In [1]:
import numpy as np
import tensorflow as tf
import json
import pickle
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from tqdm import tqdm_notebook as tqdm

In [4]:
tf.enable_eager_execution()

In [64]:
def preprocess_text(text, add_start_end=True):
    """A very basic function to preprocess the texts. Needs to be imporved."""
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    text = text.lower().strip()
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    
    
    # replacing everything with space except (a-z, A-Z, and german umlauts)
    text = re.sub(r"[^a-zA-ZäüßöÄÖÜ?]+", " ", text)
    text = text.rstrip().strip()
    
    # Removing all single letter words
    text = re.sub(r"/(^|\s+)(\S(\s+|$))+/", "", text)
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    if add_start_end:
        text = '<start> ' + text + ' <end>'
    return text

In [23]:
def get_embeddings_matrix(tokenizer, vocab_size, embedding_dim=300):
    """Loads the glove vectors, assuming there is a folder called embeddings."""
    
    # Get the word - idx dictionary
    word_index = tokenizer.word_index

    # Create a word - vector embedding dictionary (loading only the embeddings for words in our dictionary)        
    embeddings_index = {}
    with open('../embeddings/vectors.txt', 'r', encoding='utf-8') as f:
        for line in f:
            vector = line.split(' ')
            word = vector[0]
            if word in word_index:
                embeddings_index[word] = np.array(vector[1:], dtype=np.float32)

    # Create embedding matrix to laod into keras layer
    embeddings_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        if i < vocab_size:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embeddings_matrix[i] = embedding_vector
            else:
                embeddings_matrix[i] = np.random.rand(embedding_dim)
        else:
            break
    return embeddings_matrix

In [24]:
def load_single(which='Erbrecht'):
    """Loads a single dataset for testing."""
    
    data = json.load(open('../Dataset/{}.json'.format(which), 'r', encoding='utf-8'))
    questions, titles = [], []
    for item in data:
        question = preprocess_text(item['question'])
        title = preprocess_text(item['title']) 
        questions.append(question)
        titles.append(title)


    print('Maximum length of questions: ', len(max(questions, key=lambda x: len(x.split())).split()))
    print('Maximum length of titles: ', len(max(titles, key=lambda x: len(x.split())).split()))
    return questions, titles

In [56]:
def load_data(folder_path):
    '''
    Loads the JSON files puts them into a Python list (X) of strings
    where each element in the list is a single question. It also sotres the category labels
    into another list (y)
    :param folder_path: the folder path that has json file for each category containing questions and answers
    :return:
    '''
    print("loading the data ... ")
    counter = 0
    questions, titles = [], []
    for f in os.listdir(folder_path):
        if f.endswith("json"):  # read all json files
            file_path = os.path.join(folder_path, f)
            contents = json.load(open(file_path, 'r', encoding='utf-8'))
            for html_item in contents:
                questions.append(preprocess_text(html_item['question'], add_start_end=False))
                titles.append(preprocess_text(html_item['title']))
            counter = counter + 1

    print('Maximum length of questions: ', len(max(questions, key=lambda x: len(x.split())).split()))
    print('Maximum length of titles: ', len(max(titles, key=lambda x: len(x.split())).split()))
    return questions, titles

In [69]:
def tokenize_and_pad(texts, vocab_size, max_len, filters=''):
    
    # Tokenization
    tokenizer = Tokenizer(num_words=vocab_size, filters=filters)
    tokenizer.fit_on_texts(texts)

    # Create a sequence of words
    texsts_seq = tokenizer.texts_to_sequences(texts)

    # Pad words
    texsts_seq = pad_sequences(texsts_seq, maxlen=max_len, padding='post')
    
    return texsts_seq, tokenizer

## Classes

In [5]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.embedding =  tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.Bidirectional(
                    tf.keras.layers.CuDNNGRU(self.enc_units, 
                                       return_sequences=True, 
                                       return_state=True, 
                                       recurrent_initializer='glorot_uniform')
        )

    def call(self, x):
        x = self.embedding(x)
        output = self.gru(x) 
        out = output[0]
        hidden = tf.concat(output[1:], axis=-1)
        return out, hidden
    
    def initialize_hidden(self, batch_size):
        return tf.zeros((batch_size, self.enc_units))
    
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, hidden_size)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        
        self.dec_units = dec_units
        self.embedding =  tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        self.gru = tf.keras.layers.CuDNNGRU(self.dec_units, 
                                       return_sequences=True, 
                                       return_state=True, 
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output, probs=False):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)
        
        if probs:
            return tf.nn.softmax(x, axis=-1), state, attention_weights
        return x, state, attention_weights
    
class TitleGenerator:
    
    def __init__(self, max_len_questions=450, max_len_titles=10, vocab_size_q=30000,  
                 vocab_size_t=30000, embedding_dim=128, enc_units=128, dec_units=256):
        self._max_len_q = max_len_questions
        self._max_len_t = max_len_titles
        self._load(vocab_size_q, vocab_size_t, embedding_dim, enc_units, dec_units)
    
    def _load(self, vocab_size_q, vocab_size_t, embedding_dim, enc_units, dec_units):
        """Load tokenizers and model."""
        
        # ----- Load tokenizers and create word indices ----- #
        self.questions_tokenizer = pickle.load(open("tokenizers/questions_tokenizer.pkl", "rb"))
        self.titles_tokenizer = pickle.load(open("tokenizers/titles_tokenizer.pkl", "rb"))
        self.word_index_q = self.questions_tokenizer.word_index
        self.word_index_t = self.titles_tokenizer.word_index
        self.index_word_q = {v: k for k, v in self.word_index_q.items()}
        self.index_word_t = {v: k for k, v in self.word_index_t.items()}
    
        # ----- Load model and weights ----- #
        self.encoder = Encoder(vocab_size_q, embedding_dim, enc_units=enc_units)
        self.decoder = Decoder(vocab_size_t, embedding_dim, dec_units=dec_units)
        self.encoder.load_weights(os.path.join("models", "encoder_" + "bidirectional_20epochs"))
        self.decoder.load_weights(os.path.join("models", "decoder_" + "bidirectional_20epochs"))
        
    def _preprocess(self, text):
        """A very basic function to preprocess the texts. Needs to be imporved."""
          
        # Reg exp cleaning
        text = text.lower().strip()
        text = re.sub(r"([?.!,¿])", r" \1 ", text)
        text = re.sub(r'[" "]+', " ", text)
        text = re.sub(r"[^a-zA-ZäüßöÄÖÜ?]+", " ", text)
        text = text.rstrip().strip()
        text = re.sub(r"/(^|\s+)(\S(\s+|$))+/", "", text)
        
        # Tokenize, pad and convert to tensor
        inputs = self.questions_tokenizer.texts_to_sequences([text])
        inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, 
                                                           maxlen=self._max_len_q, 
                                                           padding='post') 
        inputs = tf.convert_to_tensor(inputs, dtype=tf.int32)
        return inputs
    
    def generate_title(self, raw_text):
        """Generates the title given the supplied raw text."""
        
        # Preprocess question
        processed_text = self._preprocess(raw_text)
        generated_title = ''
        enc_out, enc_hidden = self.encoder(processed_text)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([self.word_index_t['<start>']], 0)
        
        # Use the language model
        for t in range(self._max_len_t):
            predictions, dec_hidden, attention_weights = self.decoder(dec_input, dec_hidden, enc_out)
            predicted_idx = tf.argmax(predictions[0]).numpy()
            dec_input = tf.expand_dims([predicted_idx], 0) 
            generated_title += self.index_word_t[predicted_idx] + ' '
            if self.index_word_t[predicted_idx] == '<end>':
                return generated_title.replace(' <end> ', '').capitalize()
        return generated_title.replace(' <end> ', '').capitalize()

## Functions

In [409]:
def compute_loss(true, pred):
    """
    Computes the categorical cross-entropy.
    -------------
    INPUT:
    true - tf.Tensor of shape (N_batches, )           - the true index values
    pred - tf.Tensor of shape (N_batches, vocab_size) - the logits 
    -------------
    OUTPUT:
    Cross entropy between true and pred.
    """
    
    # Mask zero paddings
    mask = tf.math.logical_not(tf.math.equal(true, 0))
    
    # Compute loss
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true, logits=pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    return tf.reduce_mean(loss)

def batch_step(questions_batch, titles_batch):
    """
    Computes the loss for a single batch and does one backprop through time step.
    Assumes the encoder, decoder, optimizer, and word_indices are defined in the global scope.
    """
    
    loss = 0
    with tf.GradientTape() as tape:

        # Encode questions
        enc_output, enc_hidden = encoder(questions_batch)

        # Initial decoder hidden state is the encoder hidden state (to which attention will be applied)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([word_index_t['<start>']] * questions_batch.shape[0], 1)       

        # Teacher forcing - feeding the target as the next input
        for t in range(1, titles_batch.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += compute_loss(titles_batch[:, t], predictions)
            dec_input = tf.expand_dims(titles_batch[:, t], 1)
            
        batch_loss = (loss / int(titles_batch.shape[1]))
        
    # Update parameters
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(batch_loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

def generate_title(question, preprocess=True):
    """Generate a new title given question."""
    
    # Preprocess question
    if preprocess:
        question = preprocess_text(question, add_start_end=False)
    inputs = questions_tokenizer.texts_to_sequences([question])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, 
                                                           maxlen=max_len_questions, 
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    generated_title = ''

    enc_out, enc_hidden = encoder(inputs)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([word_index_t['<start>']], 0)
    
    for t in range(max_len_titles):
        
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_out)
        
        
        predicted_idx = tf.argmax(predictions[0]).numpy()
        generated_title += index_word_t[predicted_idx] + ' '
        dec_input = tf.expand_dims([predicted_idx], 0) # the predicted ID is fed back into the model

        if index_word_t[predicted_idx] == '<end>':
            return generated_title
    return generated_title

def validate_model(questions_batch, titles_batch):
    """Computes validation loss and outputs some generated titles."""
    
    loss = 0
    # Encode questions
    enc_output, enc_hidden = encoder(questions_batch)

    # Initial decoder hidden state is the encoder hidden state (to which attention will be applied)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([word_index_t['<start>']] * questions_batch.shape[0], 1)       

    # Teacher forcing - feeding the target as the next input
    for t in range(1, titles_batch.shape[1]):
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
        loss += compute_loss(titles_batch[:, t], predictions)
        dec_input = tf.expand_dims(titles_batch[:, t], 1)

    batch_loss = (loss / int(titles_batch.shape[1]))
    return batch_loss

def generate_titles(questions_batch, titles_batch, how_many=5, idx_to_gen=None):
    
    # Pick random number of validation questions and titles
    if idx_to_gen is None:
        idx_to_gen = np.random.permutation(questions_batch.numpy().shape[0])[:how_many]
    
    # Generate # titles
    for i in idx_to_gen:
        
        # Decode question and title
        question = " ".join([index_word_q[idx] for idx in questions_val[i].numpy() if idx != 0])
        title = " ".join([index_word_t[idx] for idx in titles_val[i].numpy() if idx!= 0])
        # Generate title
        generated_title = generate_title(question, preprocess=False)
        print('----- Title of question #{} -----'.format(i))
        print('Generated:', generated_title, 'True:', title)

# Training steps


1. Pass the input through the encoder which return encoder output and the encoder hidden state.
2. The encoder output, encoder hidden state and the decoder input (which is the start token) is passed to the decoder.
3. The decoder returns the predictions and the decoder hidden state.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use teacher forcing to decide the next input to the decoder.
6. Teacher forcing is the technique where the target word is passed as the next input to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

## Load raw questions, preprocess, tokenize and save

In [218]:
# questions, titles = load_data('../Dataset/')

# embeddings_q = get_embeddings_matrix(questions_tokenizer, vocab_size_q, embedding_dim=embedding_dim)
# embeddings_t = get_embeddings_matrix(titles_tokenizer, vocab_size_t, embedding_dim=embedding_dim)

# questions_seq, questions_tokenizer = tokenize_and_pad(questions, vocab_size_q, max_len=max_len_questions, filters="")
# titles_seq, titles_tokenizer = tokenize_and_pad(titles, vocab_size_t, max_len=max_len_titles, filters="")

# # Save sequence data and tokenizers
# np.save("questions_seq.npy", questions_seq)
# np.save("titles_seq.npy", titles_seq)
 
# pickle.dump(questions_tokenizer, open('questions_tokenizer.pkl', 'wb'))
# pickle.dump(titles_tokenizer, open('titles_tokenizer.pkl', 'wb'))

## Load tokenized questions and titles

In [234]:
questions_seq = np.load("questions_seq.npy")
titles_seq = np.load("titles_seq.npy")

questions_tokenizer = pickle.load(open("questions_tokenizer.pkl", "rb"))
titles_tokenizer = pickle.load(open("titles_tokenizer.pkl", "rb"))

word_index_q = questions_tokenizer.word_index
word_index_t = titles_tokenizer.word_index
index_word_q = {v: k for k, v in word_index_q.items()}
index_word_t = {v: k for k, v in word_index_t.items()}


questions_seq, questions_val, titles_seq, titles_val = train_test_split(questions_seq, titles_seq, test_size=0.001)

print(questions_seq.shape)
print(titles_seq.shape)
print(questions_val.shape)
print(titles_val.shape)


questions_val = tf.convert_to_tensor(questions_val, dtype=tf.int32)
titles_val = tf.convert_to_tensor(titles_val, dtype=tf.int32)

## Model and training hyperparameters

In [414]:
# Trainiing parameters
batch_size = 128
learning_rate = 0.005
epochs = 20
enc_units = 128
dec_units = 256
max_len_questions = 450
max_len_titles = 10
vocab_size_q = 30000
vocab_size_t = 30000
embedding_dim = 128

## Create and train model

In [412]:
encoder = Encoder(vocab_size_q, embedding_dim, enc_units=enc_units)
decoder = Decoder(vocab_size_t, embedding_dim, dec_units=dec_units)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
dataset = tf.data.Dataset.from_tensor_slices((questions_seq, titles_seq))
dataset = dataset.shuffle(questions_seq.shape[0]).batch(batch_size)

### Train for 20 epochs

In [415]:
for epoch in range(1, epochs + 1):
    with tqdm(total=questions_seq.shape[0] // batch_size, desc='Epoch {}'.format(epoch)) as p_bar:
        for i, (questions_batch, titles_batch) in enumerate(dataset):
            batch_loss = batch_step(questions_batch, titles_batch)
            p_bar.update(1)
            p_bar.set_postfix_str("Training loss: {0:.3f}".format(batch_loss.numpy()))
        
    # After end of epoch, validate and generate some titles
    val_loss = validate_model(questions_val, titles_val)
    print('Epoch', epoch, 'Validation loss:', val_loss)
    generate_titles(questions_val, titles_val, how_many=10)


----- Title of question #85 -----
Generated: unterhalt für die mutter <end>  True: <start> hartz ablehnung wegen verwertbaren vermögen <end>
----- Title of question #31 -----
Generated: ladendiebstahl <end>  True: <start> wiederholter ladendiebstahl verbeamtung ? <end>
----- Title of question #55 -----
Generated: rücktritt vom kaufvertrag <end>  True: <start> neuwagenkauf über freien vermittler <end>
----- Title of question #78 -----
Generated: ebay kleinanzeigen <end>  True: <start> pkw haftpflicht schaden schutzplanke unberechtigte forderungen <end>
----- Title of question #22 -----
Generated: ebay kleinanzeigen <end>  True: <start> wegerecht täuschung bei kaufvertrag <end>
----- Title of question #4 -----
Generated: kündigung <end>  True: <start> wahl des beirates einer weg <end>
----- Title of question #59 -----
Generated: <end>  True: <start> rückbau von pv anlagen auf in hessen ? <end>
----- Title of question #32 -----
Generated: ebay kleinanzeigen <end>  True: <start> auto war 


----- Title of question #27 -----
Generated: mietvertrag mit wohnung <end>  True: <start> kostenübernahme für übernommene bauliche änderungen rechtens ? <end>
----- Title of question #50 -----
Generated: ladendiebstahl <end>  True: <start> strafmaß bedrohung und beleidigung gegen firma <end>
----- Title of question #19 -----
Generated: unterhalt für volljährige kinder <end>  True: für vergangene zeiten wenn sie sich von nun an <end>
----- Title of question #31 -----
Generated: ladendiebstahl ladendiebstahl <end>  True: <start> wiederholter ladendiebstahl verbeamtung ? <end>
----- Title of question #13 -----
Generated: kündigung durch arbeitnehmer <end>  True: <start> urlaubsabgeltung bei kündigung in probezeit nach krankheit <end>
----- Title of question #14 -----
Generated: <end>  True: <start> variabler gehaltsbestandteil aber ziele nicht definiert vergessen <end>
----- Title of question #94 -----
Generated: kindesunterhalt bei kindesunterhalt <end>  True: <start> zuzahlung fürs pfl


----- Title of question #32 -----
Generated: kfz verkauf <end>  True: <start> auto war totalschaden <end>
----- Title of question #80 -----
Generated: kündigung während probezeit <end>  True: <start> eigene kündigung zum ende der elternzeit <end>
----- Title of question #54 -----
Generated: mietrecht <end>  True: <start> nachvermietung gewerblicher räume <end>
----- Title of question #45 -----
Generated: widerruf eines <end>  True: <start> widerruf handyvertrag <end>
----- Title of question #63 -----
Generated: unterhalt für kind <end>  True: <start> unterhalt an volljährigen sohn durfte er ausziehen ? <end>
----- Title of question #21 -----
Generated: vermieter verlangt mieter <end>  True: mich der vermieter zur renovierung vor einzug erpressen ? <end>
----- Title of question #16 -----
Generated: befristeter arbeitsvertrag und kündigung <end>  True: <start> zeit und sachbefristung <end>
----- Title of question #90 -----
Generated: einkommensteuer für einkommenssteuer <end>  True: <st


----- Title of question #58 -----
Generated: beamtenrecht <end>  True: <start> arbeitsunfähigkeitsbescheinigung <end>
----- Title of question #5 -----
Generated: fristlose kündigung wegen kündigung <end>  True: fristlose kündigung um zu wandeln in eine fristgerechte ? <end>
----- Title of question #86 -----
Generated: erbrecht <end>  True: <start> erbfolge brd <end>
----- Title of question #51 -----
Generated: bewährung wegen betrugs <end>  True: <start> betrug vor einer verurteilung <end>
----- Title of question #64 -----
Generated: mietrecht auszug <end>  True: <start> mietvertrag renovierung und mietminderung ? <end>
----- Title of question #24 -----
Generated: kündigung durch ag <end>  True: <start> sind meine kündigungen rechtlich wirksam oder nicht ? <end>
----- Title of question #75 -----
Generated: unterschlagung ? <end>  True: <start> konto meiner frau aufgelöst <end>
----- Title of question #8 -----
Generated: ebay verkäufer will nicht zurück <end>  True: <start> privatkauf 


----- Title of question #64 -----
Generated: küche vermieter <end>  True: <start> mietvertrag renovierung und mietminderung ? <end>
----- Title of question #85 -----
Generated: alg ii und alg ii <end>  True: <start> hartz ablehnung wegen verwertbaren vermögen <end>
----- Title of question #18 -----
Generated: krankenversicherung nach deutschland <end>  True: <start> wer übernimmt krankheitskosten bei entsendung des arbeitnehmers ? <end>
----- Title of question #52 -----
Generated: testament und testament <end>  True: <start> brauche vergleichsurteile zwecks testamentsüberprüfung stück <end>
----- Title of question #63 -----
Generated: unterhalt für volljährige tochter <end>  True: <start> unterhalt an volljährigen sohn durfte er ausziehen ? <end>
----- Title of question #1 -----
Generated: ladendiebstahl und <end>  True: <start> eintragung in polizeiliches führungszeugniss <end>
----- Title of question #33 -----
Generated: urlaubsanspruch bei kündigung <end>  True: <start> urlaubsansp

...


----- Title of question #87 -----
Generated: wechsel in gkv familienversicherung <end>  True: <start> wechsel in die gkv als rentner ? <end>
----- Title of question #61 -----
Generated: führerschein in deutschland <end>  True: <start> hallo fs ohne mpu ? <end>
----- Title of question #4 -----
Generated: notarrechnung <end>  True: <start> wahl des beirates einer weg <end>
----- Title of question #2 -----
Generated: unterhalt für unterhalt <end>  True: behörde mir von meinem einkommen sozialhilfe wegnehmen kindesunterhalt ? <end>
----- Title of question #75 -----
Generated: strafanzeige wegen harz <end>  True: <start> konto meiner frau aufgelöst <end>
----- Title of question #36 -----
Generated: kostenvoranschlag für kostenvoranschlag auftrag <end>  True: <start> kostenvoranschlag <end>
----- Title of question #68 -----
Generated: vererben von erbschaft <end>  True: <start> pflichtteilsanspruch mit bedingung ? <end>
----- Title of question #71 -----
Generated: rückerstattung von privat 


----- Title of question #75 -----
Generated: ich mich aus meiner frau <end>  True: <start> konto meiner frau aufgelöst <end>
----- Title of question #71 -----
Generated: fragen an der auslöse ? <end>  True: welche kosten müssen beim erfolgten rücktritt erstattet werden ? <end>
----- Title of question #58 -----
Generated: bei mehreren und bei nebenberuflich waren <end>  True: <start> arbeitsunfähigkeitsbescheinigung <end>
----- Title of question #45 -----
Generated: widerruf der widerrufsfrist <end>  True: <start> widerruf handyvertrag <end>
----- Title of question #73 -----
Generated: doppelte anmeldung bei hartz <end>  True: <start> hartz mietrecht <end>
----- Title of question #59 -----
Generated: in der verauslagten <end>  True: <start> rückbau von pv anlagen auf in hessen ? <end>
----- Title of question #5 -----
Generated: kündigung durch arbeitgeber <end>  True: fristlose kündigung um zu wandeln in eine fristgerechte ? <end>
----- Title of question #9 -----
Generated: weg weg bes


----- Title of question #18 -----
Generated: in der gesetzlichen jahren in deutschland bei umzug ins ausland  True: <start> wer übernimmt krankheitskosten bei entsendung des arbeitnehmers ? <end>
----- Title of question #43 -----
Generated: <end>  True: <start> befreiung von beibringung des <end>
----- Title of question #32 -----
Generated: unfallschaden beim autokauf unfall <end>  True: <start> auto war totalschaden <end>
----- Title of question #29 -----
Generated: ehevertrag ehevertrag ehevertrag <end>  True: <start> ehevertrag ungülgitg gütertrennungsklausel weiterhin gültig ? <end>
----- Title of question #61 -----
Generated: führerschein in der eu führerschein <end>  True: <start> hallo fs ohne mpu ? <end>
----- Title of question #95 -----
Generated: strafanzeige gegen unterlassungserklärung <end>  True: <start> strafanzeige wegen internetveröffentlichung gegen einen verein <end>
----- Title of question #35 -----
Generated: angebot einer <end>  True: <start> differenz zwischen a


----- Title of question #89 -----
Generated: für wechsel von <end>  True: <start> durch jobagentur <end>
----- Title of question #29 -----
Generated: ehevertrag ehevertrag ehevertrag <end>  True: <start> ehevertrag ungülgitg gütertrennungsklausel weiterhin gültig ? <end>
----- Title of question #53 -----
Generated: nach btmg <end>  True: <start> übernahme des grundstückes ohne abgrenzungen <end>
----- Title of question #70 -----
Generated: schwerbehinderung und <end>  True: bbig voraussetzungen für einen praktikumsplatz steuer bei einem rechtsanwalt <end>
----- Title of question #17 -----
Generated: erbschaft einbehalten ? <end>  True: jetzt machen damit ich die erbschaft ausschlagen kann ? <end>
----- Title of question #10 -----
Generated: anzeige wegen verleumdung am arbeitsplatz <end>  True: <start> verleumdung und rufmord <end>
----- Title of question #25 -----
Generated: zugewinn bei der geschäftsgrundlage im rahmen der <end>  True: güterrecht ansprüche nicht geltend gemacht inde


----- Title of question #27 -----
Generated: mietvertrag ohne <end>  True: <start> kostenübernahme für übernommene bauliche änderungen rechtens ? <end>
----- Title of question #7 -----
Generated: teilzeit während elternzeit <end>  True: <start> teilzeit in elternzeit änderungsvertrag oder ergänzungsvertrag <end>
----- Title of question #90 -----
Generated: freiwillige steuererklärung <end>  True: <start> besteuerung rente aus singapur <end>
----- Title of question #69 -----
Generated: alg kindergeld alg mit partner <end>  True: <start> tochter bekommt nach ausbildung alg <end>
----- Title of question #25 -----
Generated: zugewinn erbschein als die befangenheit im familiengericht ? <end>  True: güterrecht ansprüche nicht geltend gemacht indexierung a passiva anwaltshaftung <end>
----- Title of question #75 -----
Generated: anzeige wegen verleumdung <end>  True: <start> konto meiner frau aufgelöst <end>
----- Title of question #32 -----
Generated: fahrzeug unfallschaden entdeckt unfalls

### Train for 20 more epochs

In [441]:
for epoch in range(1, epochs + 1):
    with tqdm(total=questions_seq.shape[0] // batch_size, desc='Epoch {}'.format(epoch)) as p_bar:
        for i, (questions_batch, titles_batch) in enumerate(dataset):
            batch_loss = batch_step(questions_batch, titles_batch)
            p_bar.update(1)
            p_bar.set_postfix_str("Training loss: {0:.3f}".format(batch_loss.numpy()))
        
    # After end of epoch, validate and generate some titles
    val_loss = validate_model(questions_val, titles_val)
    print('Epoch', epoch, 'Validation loss:', val_loss.numpy())
    generate_titles(questions_val, titles_val, how_many=10)

In [6]:
title_gen = TitleGenerator()

In [416]:
# Save 
# encoder.save_weights(os.path.join("models", "encoder_" + "bidirectional_20epochs"))
# decoder.save_weights(os.path.join("models", "decoder_" + "bidirectional_20epochs"))

In [7]:
raw_text = """ Im Jahre 1998 wurde ich zu 6 Monaten Haft auf Bewährung verurteilt. Grund war Besitz und Verbreitung kinderpornografischer Schriften. Zusätzlich erhielt ich eine Auflage des Verbots der Betreuung und Beaufsichtigung von Kindern und Jugendlichen (oder so ähnlich). Im normalen Führungszeugnis wurde das ja schon längst nicht mehr ausgewiesen. Wie verhält es sich beim erweiterten Führungszeugnis. Taucht hier noch irgendetwas auf bzw. wann wurde das ggf. getilgt? """

In [8]:
title_gen.generate_title(raw_text)

'Erweitertes führungszeugnis'

In [439]:
validate_model(questions_val, titles_val)

<tf.Tensor: id=318648821, shape=(), dtype=float32, numpy=3.7262657>

In [ ]:
title_gen.generate_title(raw_text)

### Train with dropout

In [ ]:
encoder = Encoder(vocab_size_q, embedding_dim, enc_units=enc_units)
decoder = Decoder(vocab_size_t, embedding_dim, dec_units=dec_units)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
dataset = tf.data.Dataset.from_tensor_slices((questions_seq, titles_seq))
dataset = dataset.shuffle(questions_seq.shape[0]).batch(batch_size)